# Cluster Data Extraction

Processes all LArIAT events to extract clusters from connected regions in both collection and induction planes. Creates a structured dataset where each cluster becomes a data point with properties like area, intensity, geometry, and transformed matrices for analysis.

**Output:** DataFrame with cluster features for machine learning and physics analysis.

In [72]:
import sys
sys.path.append('..')

In [73]:
import uproot

import numpy as np
import pandas as pd
import awkward as ak
import matplotlib.pyplot as plt
import seaborn as sns

from skimage.measure import label, regionprops
from scipy.ndimage import label as scipy_label
from collections import deque

from lariat import Event

In [4]:
deuterons = pd.read_csv('/Users/user/data/research/proton-deuteron/csv/deuteron_candidates_bbox_t100.csv') # from within vertices bounding box
protons = pd.read_csv('/Users/user/data/research/proton-deuteron/csv/protons_one_track_filepaths.csv') # with only one track from reco

In [ ]:
def extract_all_clusters_to_df(events_df, particle_type, threshold=15, max_events=None):
    """Extract all clusters from events and create a pandas dataframe"""
    
    cluster_data = []
    
    # Limit events if specified
    if max_events:
        events_df = events_df.head(max_events)
    
    for i, row in events_df.iterrows(): # iterrows gives index, Series (row)
        try:
            # Create event
            event = Event(row.file_path, index=row.event_index)
            
            # Get connected regions for collection plane
            clabeled, cregions = event.connectedregions(event.collection, threshold=threshold)
            ilabeled, iregions = event.connectedregions(event.induction, threshold=threshold)
            
            # Process collection plane clusters
            if cregions is not None:
                for j, region in enumerate(cregions): # index, element 

                    matrix = region.image_intensity 
                    matrix_transformed = matrix.T[::-1] # image of cluster 
                    column_maxes = np.max(matrix_transformed, axis=0) # 1D matrix, max ADC for each wire in cluster - gives a 1D view of energy deposition
                    
                    cluster_info = {
                        'event_idx': i,
                        'run': row.run,
                        'subrun': row.subrun,
                        'event': row.event,
                        'file_path': row.file_path,
                        'event_index': row.event_index,
                        'particle_type': particle_type,
                        'plane': 'collection',
                        'cluster_idx': j,
                        'area': region.area,
                        'max_intensity': region.intensity_max,
                        'min_intensity': region.intensity_min,
                        'mean_intensity': region.intensity_mean,
                        'total_intensity': region.intensity_image.sum(),
                        'centroid_x': region.centroid[0],
                        'centroid_y': region.centroid[1],
                        'bbox_min_row': region.bbox[0],
                        'bbox_min_col': region.bbox[1],
                        'bbox_max_row': region.bbox[2],
                        'bbox_max_col': region.bbox[3],
                        'width': region.bbox[3] - region.bbox[1],
                        'height': region.bbox[2] - region.bbox[0],
                        'aspect_ratio': (region.bbox[3] - region.bbox[1]) / (region.bbox[2] - region.bbox[0]),
                        'compactness': region.area / ((region.bbox[3] - region.bbox[1]) * (region.bbox[2] - region.bbox[0])),
                        'image_intensity': region.image_intensity,  # Original image
                        'matrix_transformed': matrix_transformed,   # Transposed and flipped matrix
                        'column_maxes': column_maxes               # Column maxes array
                    }
                    cluster_data.append(cluster_info)
            
            # Process induction plane clusters
            if iregions is not None:
                for j, region in enumerate(iregions):
                    # Get the matrix and column maxes
                    matrix = region.image_intensity
                    matrix_transformed = matrix.T[::-1]
                    column_maxes = np.max(matrix_transformed, axis=0)
                    
                    cluster_info = {
                        'event_idx': i,
                        'run': row.run,
                        'subrun': row.subrun,
                        'event': row.event,
                        'file_path': row.file_path,
                        'event_index': row.event_index,
                        'particle_type': particle_type,
                        'plane': 'induction',
                        'cluster_idx': j,
                        'area': region.area,
                        'max_intensity': region.intensity_max,
                        'min_intensity': region.intensity_min,
                        'mean_intensity': region.intensity_mean,
                        'total_intensity': region.intensity_image.sum(),
                        'centroid_x': region.centroid[0],
                        'centroid_y': region.centroid[1],
                        'bbox_min_row': region.bbox[0],
                        'bbox_min_col': region.bbox[1],
                        'bbox_max_row': region.bbox[2],
                        'bbox_max_col': region.bbox[3],
                        'width': region.bbox[3] - region.bbox[1],
                        'height': region.bbox[2] - region.bbox[0],
                        'aspect_ratio': (region.bbox[3] - region.bbox[1]) / (region.bbox[2] - region.bbox[0]),
                        'compactness': region.area / ((region.bbox[3] - region.bbox[1]) * (region.bbox[2] - region.bbox[0])),
                        'image_intensity': region.image_intensity,  # Original image
                        'matrix_transformed': matrix_transformed,   # Transposed and flipped matrix
                        'column_maxes': column_maxes               # Column maxes array
                    }
                    cluster_data.append(cluster_info)
                    
        except Exception as e:
            print(f"Error processing event {i}: {e}")
            continue
    
    return pd.DataFrame(cluster_data)

In [ ]:

# deuteron_clusters_df = extract_all_clusters_to_df(deuterons, 'deuteron', threshold=15)

# proton_clusters_df = extract_all_clusters_to_df(protons, 'proton', threshold=15)

# all_clusters_df = pd.concat([deuteron_clusters_df, proton_clusters_df], ignore_index=True)

# all_clusters_df.to_pickle('/Users/user/data/research/proton-deuteron/csv/all_clusters_df.pkl')